# Computer statistical shape model in fundamental coordinates representation

In [ ]:
import numpy as np
import pyvista as pv

from morphomatics.geom import Surface
from morphomatics.stats import StatisticalShapeModel
from morphomatics.manifold import FundamentalCoords

to_surf = lambda mesh: Surface(mesh.points, mesh.faces.reshape(-1, 4)[:, 1:])
S1 = to_surf(pv.read('./data/straight.obj'))
S2 = to_surf(pv.read('./data/helix.obj'))

SSM = StatisticalShapeModel(lambda ref: FundamentalCoords(ref))
SSM.construct([S1,S2])
print(f'variances: {SSM.variances}')

pl = pv.Plotter(notebook=True)
f = np.hstack([3 * np.ones(len(SSM.mean.f), dtype=int).reshape(-1, 1), SSM.mean.f])
mesh = pv.PolyData(SSM.mean.v, f)
pl.add_mesh(mesh)
pl.reset_camera()

std = np.sqrt(SSM.variances[0])
update_mesh = lambda a: pl.update_coordinates(
SSM.space.from_coords(SSM.space.exp(SSM.mean_coords, a * std * SSM.modes[0])))

## Server-side rendering with `ipyvtklink`.

In [ ]:
slider = pl.add_slider_widget(callback=update_mesh, rng=(-1, 1))
pl.show(jupyter_backend='ipyvtklink')

## Client-side rendering with `ipygany` and `ipywidgets` for interaction.

In [ ]:
from ipywidgets import interact

@interact
def plot(x=(-1,1,.1)):
    update_mesh(x)
    pl.show(jupyter_backend='ipygany')